In [ ]:
import pandas as pd 
import re 

In [ ]:
url = 'https://drive.google.com/file/d/130KYOX8O4wrP_T8vdz2GfvJRQ03ONmE7/view?usp=share_link'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
df = pd.read_csv(path, index_col =[0])
df.head(2)

,ID объявления,Количество комнат,Тип,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Телефоны,...,Окна,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,Серия дома,"Высота потолков, м",Лифт,Мусоропровод,Ссылка на объявление
0,271271157,4,Квартира,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79166369231,...,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.0,"Пасс (4), Груз (1)",Да,https://www.cian.ru/rent/flat/271271157
1,271634126,4,Квартира,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79850608590,...,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.5,"Пасс (1), Груз (1)",Нет,https://www.cian.ru/rent/flat/271634126


In [ ]:
df = df[df['Адрес'].str.startswith('Москва')]
df.reset_index(inplace = True)

In [ ]:
# Сохраняем датасет без региональных объявлений
df.to_csv('data_cleaned.csv')

ID

In [ ]:
#переименовали колонку с ID объявления, Тип
df.rename({'ID  объявления': 'ID', 'Тип': 'Type'}, axis='columns', inplace = True)

Количество комнат

In [ ]:
# Обрабатываем колонку с количеством комнат, разделяем на число комнат, планировка. number_of_rooms, rooms_plan  
df[['number_of_rooms','rooms_plan']] = df['Количество комнат'].apply(lambda x: pd.Series(str(x).split(", ")))
# Заполняем нан в  number_of_rooms, rooms_plan
max_room_count = int(df['number_of_rooms'].value_counts(dropna = False).idxmax())
df['number_of_rooms'] = df['number_of_rooms'].map(lambda x: max_room_count if  x == 'nan' else x)
df['rooms_plan'] = df['rooms_plan'].map(lambda x:  x if  x == x else 'Неизвестно')

Площадь квартиры

In [ ]:
# Обрабатываем колонку с площадью квартиры, берём только общую площадь
df['flat_area'] = [row.split('/')[0] for row in df['Площадь, м2']]
#потом  дроп старой колонки df = df.drop(['Площадь, м2'], axis = 1) 

Дом (этаж,этажность тип)

In [ ]:
#Дом. разбиваем колонку. flat_floor,max_floor,house_type
df['flat_floor'] = [int(row.split('/')[0]) for row in df['Дом']]
df['max_floor'] = [int(row.split('/')[1].split(',')[0]) for row in df['Дом']]
df['house_type'] = [row.split(', ')[-1] if len(row.split(', ')[-1]) > 6 else 'Неизвестно' for row in df['Дом']]
#а потом дропнем  старый Дом)))# df = df.drop(['Дом'], axis = 1)


In [ ]:
# дропнуть Площадь комнат. Невозможно адекватно разделиь данные
# df = df.drop(['Площадь комнат'], axis = 1)


# Лена

# Телефоны

In [ ]:
# Разделяем телефоны на два столбца, чтоб проще проводить фильтрацию
df['phone_1'] = 0
df['phone_2'] = 0
for index in range(len(df)):
    phones = df['Телефоны'][index].split(',')
    df['phone_1'][index] = phones[0]
    if len(phones)==2:
        df['phone_2'][index] = phones[1] 
    else:
        continue

<ipython-input-43-c2295f0a6c88>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['phone_1'][index] = phones[0]
<ipython-input-43-c2295f0a6c88>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['phone_2'][index] = phones[1]


In [ ]:
df.drop(columns = 'Телефоны', inplace = True)

#Балконы

In [ ]:
# Заменяем все значения Nan на 0, т.к. отсутсвие данных скорее всего говорит об отсуствие балкона
df['Балкон'].fillna(0, inplace =True) 

# создаем новые столбцы, куда будме вносить данные по количеству балконов в зависимости от их типа
df['Balcony'] = 0
df['Loggia'] = 0
df['No_balcony'] = 0

# заполняем  столбцы в зависимости
for index in range(len(df)):
    
    if df["Балкон"][index] !=0:
        
        if 'Балкон' in df["Балкон"][index] and 'Лоджия' in df["Балкон"][index]: # проверка на оба типа
            df['Balcony'][index] = df["Балкон"][index].split(', ')[0].split(' ')[1].replace('(', '').replace(')','')
            df['Loggia'][index] = df["Балкон"][index].split(', ')[1].split(' ')[1].replace('(', '').replace(')','')
        
        elif 'Балкон' in df["Балкон"][index]:
            df['Balcony'][index]=df["Балкон"][index].split(' ')[1].replace('(', '').replace(')','')
        
        elif 'Лоджия' in df["Балкон"][index]:
            df['Loggia'][index]=df["Балкон"][index].split(' ')[1].replace('(', '').replace(')','')
    
    else:
        df['No_balcony'][index] = 1

In [ ]:
df.drop(columns = 'Балкон', inplace = True)

# Высота потолков

In [ ]:
false_ceil = df['Высота потолков, м'][df['Высота потолков, м'] > 5]
false_ceil
# из всех "ошибочных" значений, все подчиняются закономерности (трехзначные цифры в мм, двухзначные в см, 
# остальное просто высокие потолки *звуки зависти*), кроме одного значения с индексом 6016

In [ ]:
df.loc[6016]
# изучив объявление (по ссылке), делаем вывод, что скорее всего опечатка
df.loc[6016]['Высота потолков, м'] = 2.8
df['Ceiling_height'] = 0
for index in range(len(df)):
    try:
        if 20<=round(df["Высота потолков, м"][index])<=99:
            df['Ceiling_height'][index] = df["Высота потолков, м"][index]/10
        elif round(df["Высота потолков, м"][index])>99:
            df['Ceiling_height'][index] = df["Высота потолков, м"][index]/100
        else:
            df['Ceiling_height'][index] = df["Высота потолков, м"][index]
    except ValueError:
        df['Ceiling_height'][index] = df['Высота потолков, м'].median()

Streaming output truncated to the last 5000 lines.
<ipython-input-48-2b35357a8d36>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Ceiling_height'][index] = df["Высота потолков, м"][index]
<ipython-input-48-2b35357a8d36>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Ceiling_height'][index] = df['Высота потолков, м'].median()
<ipython-input-48-2b35357a8d36>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Ceiling_heig

In [ ]:
df.drop(columns = 'Высота потолков, м', inplace = True)

# Лифт

In [ ]:
# создаем новые столбцы для количества пассажирских, грузовых и отсуствующих лифтов
df['Elevator_pass'] = 0
df['Elevator_cargo'] = 0
df['Elevator_no'] = 0

# для всех домов с количеством этажей не более 5 присваиваем 1 в Elevator_no (стандарт):
df['Elevator_no'][df[['max_floor', 'Лифт']].query('max_floor<=5')['Лифт'].isnull().index] = 1

# для всех домов с количеством этажей от 6 до 9 присваиваем 1 пассажирский (опять же по стандартам):
df['Elevator_pass'][df[['max_floor', 'Лифт']].query('5<max_floor<=9')['Лифт'].isnull().index] = 1

# для всех остальных домов присваиваем 1 пассажирский и 1 грузовой (опять же по стандартам)
df['Elevator_pass'][df[['max_floor', 'Лифт']].query('max_floor>9')['Лифт'].isnull().index] = 1
df['Elevator_cargo'][df[['max_floor', 'Лифт']].query('max_floor>9')['Лифт'].isnull().index] = 1

for index in range(len(df)):
    if df["Лифт"][index] == df["Лифт"][index]: # проверка на сортировку от NaN
        
        if 'Пасс' in df["Лифт"][index] and 'Груз' in df["Лифт"][index]: #Если оба случая, то Пасс будет первым
            df['Elevator_pass'][index] = df["Лифт"][index].split(', ')[0].split(' ')[1].replace('(', '').replace(')','')
            df['Elevator_cargo'][index] = df["Лифт"][index].split(', ')[1].split(' ')[1].replace('(', '').replace(')','')
        elif 'Пасс' in df["Лифт"][index]:
            df['Elevator_pass'][index]=df["Лифт"][index].split(' ')[1].replace('(', '').replace(')','')
        else:
            df['Elevator_cargo'][index]=df["Лифт"][index].split(' ')[1].replace('(', '').replace(')','')
        
    else:
        continue

<ipython-input-50-2b81089012ec>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Elevator_no'][df[['max_floor', 'Лифт']].query('max_floor<=5')['Лифт'].isnull().index] = 1
<ipython-input-50-2b81089012ec>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Elevator_pass'][df[['max_floor', 'Лифт']].query('5<max_floor<=9')['Лифт'].isnull().index] = 1
<ipython-input-50-2b81089012ec>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df

In [ ]:
df.drop(columns = 'Лифт', inplace = True)

# Мусоропровод

In [ ]:
# скорее всего отсутствие информации говорит об отсуствии предмета
df['Мусоропровод'].fillna('Нет')
df['carbage'] = df['Мусоропровод'].map(lambda x:  1 if  x =='да' else 0)
df.drop(columns = 'Мусоропровод', inplace = True)

# Окна


In [ ]:
# Можно с учетом планировки (площадь квартиры+количество комнат), но нет времени, поэтому заполню модой
df['Окна'].fillna(method='bfill', inplace = True)
df = pd.concat([df, pd.get_dummies(df['Окна'])], axis = 1)
df.rename(columns = {'Во двор' : 'to_yard', 
                     'На улицу' : 'to_street', 
                     'На улицу и двор' : 'to_yard_and_street'}, inplace = True)
df.drop(columns = 'Окна', inplace = True)

Парковка: да /нет

In [ ]:
# парковка ремонт parking renovation
df.rename({'Парковка': 'parking_lot', 'Ремонт': 'renovation'}, axis='columns', inplace=True)
# parking_list = ['подземная', 'наземная', 'открытая', 'многоуровневая', 'на крыше']
df['parking_lot'] = df['parking_lot'].fillna(0)
df['parking_lot'] = df['parking_lot'].map(lambda x:  x if  x ==0 else 1)
df[['parking_lot','renovation']]


,parking_lot,renovation
0,1,Дизайнерский
1,1,Дизайнерский
2,1,Евроремонт
3,1,Евроремонт
4,0,Евроремонт
...,...,...
19732,0,Евроремонт
19733,0,Евроремонт
19734,0,Дизайнерский
19735,1,Евроремонт


Ремонт. Пропуски заполняем как отсутсвие ремонта изз-за того,что это важный параметр, котрый повлияет на стоимость аренды

In [ ]:
# заполним нан как отсутсвие ремонта
df['renovation'] = df['renovation'].fillna(0)
df['renovation'] = df['renovation'].map(lambda x:  x if  x ==0 else 1)

# Егор

## Чищу Метро

PS раскомментировать мои можно ctrl+k+c

In [ ]:
df['Метро'].fillna('a (b c', inplace = True)
df['Метро'] = df['Метро'].str.replace(')', '').str.split('(')

<ipython-input-56-40a2f54eeecd>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Метро'] = df['Метро'].str.replace(')', '').str.split('(')


In [ ]:
df_metro = pd.DataFrame(df["Метро"].to_list(), columns=['Station', 'Time1'])
df_metro["Time1"] = df_metro["Time1"].str.split(' ', 1)

df_metro_1 = pd.DataFrame(df_metro["Time1"].to_list(), columns=['Time', 'Vehicle'])

In [ ]:
df_metro = pd.concat([df_metro, df_metro_1], axis = 1)
df_metro.drop('Time1', axis = 1, inplace = True)
df_metro['Time'].replace(['b', 'None'], -1, inplace = True)

In [ ]:
df_metro['Vehicle'].replace(['c', 'None'], None, inplace = True)
d1 = pd.get_dummies(df_metro['Vehicle'])
d1.rename(columns = ({'мин на машине' : 'Car', 'мин пешком' : 'On Foot'}), inplace = True)
df_metro = pd.concat([d1, df_metro], axis = 1)
df_metro.drop(['Station', 'Vehicle'], axis = 1, inplace = True)

df_metro['Time'] = df_metro['Time'].apply(lambda x: int(x) if x != None else x)
car_max = df_metro['Time'][df_metro['Car'] == 1].max()
foot_max = df_metro['Time'][df_metro['On Foot'] == 1].max()

for index, row in df_metro.iterrows():
    if row[0] == 1 and row[2] == -1:
        row[2] = car_max
    if row[1] == 1 and row[2] == -1:
        row[2] = foot_max

## Чищу Цену 

In [ ]:
price = df['Цена']
splt = price.apply(lambda x: x.split(','))
rent = price.apply(lambda x: x.split(',')[0]).apply(lambda x: float(''.join(
    re.findall("\d+\.\d+", x))))

deposit = price.apply(lambda x: x.split(',')[1]).apply(lambda x: ''.join(re.findall("\d+", x))).apply(
                    lambda x: float(x) if x != '' else 0)

In [ ]:
rent.rename('Rent', inplace = True)
deposit.rename('Deposit', inplace = True)
price_df = pd.concat([rent, deposit], axis = 1)

## Чищу Дети/Животные 

In [ ]:
anchil_df = pd.get_dummies(df['Можно с детьми/животными'])

anchil_df.rename(columns = {'Можно с детьми' : 'Children', 'Можно с детьми, Можно с животными' :
                  'Children_Animals', 'Можно с животными' : 'Animals'}, inplace = True)

## Чищу Дополнительно  

In [ ]:
dop_df = df['Дополнительно'].apply(lambda x: x.split(', ') if x == x else x).fillna('d')
dop_df = pd.get_dummies(dop_df.apply(pd.Series).stack()).sum(level=0)
dop_df.rename(columns = ({'d' : 'NaN', 'Ванна' : 'Bathtab', 'Душевая кабина' : 'Shower Cabin',
                         'Интернет' : 'Internet', 'Кондиционер' : 'Air Conditioner', 'Мебель в комнатах' : 
                         'Furniture Rooms', 'Мебель на кухне' : 'Furniture Kitchen',
                         'Посудомоечная машина' : 'Dishwasher', 'Стиральная машина' : 'Washing Machine',
                         'Телевизор' : 'TV', 'Телефон' : 'Telephone', 'Холодильник' : 'Fridge'}),
              inplace = True)
dop_df.drop('NaN', axis = 1, inplace = True)

<ipython-input-63-dcae3a2559a3>:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  dop_df = pd.get_dummies(dop_df.apply(pd.Series).stack()).sum(level=0)


## Чищу Санузел))


In [ ]:
toil = df['Санузел']
toil_df = toil.str.split(',')
toil_df.fillna('e', inplace = True)
toil_df = pd.get_dummies(toil_df.apply(pd.Series).stack()).sum(level=0)

<ipython-input-64-ce737d261657>:4: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  toil_df = pd.get_dummies(toil_df.apply(pd.Series).stack()).sum(level=0)


In [ ]:
al = toil_df.filter(like='Раздельный (1)')
be = toil_df.filter(like='Раздельный (2)')
ga = toil_df.filter(like='Раздельный (3)')
de = toil_df.filter(like='Раздельный (4)')

sep_1 = pd.Series(al.iloc[:, 0] | al.iloc[:, 1], name = 'Sep_Toil_1')
sep_2 = pd.Series(be.iloc[:, 0] | be.iloc[:, 1], name = 'Sep_Toil_2')
sep_3 = pd.Series(ga.iloc[:, 0] | ga.iloc[:, 1], name = 'Sep_Toil_3')
sep_4 = pd.Series(de.iloc[:, 0] | de.iloc[:, 1], name = 'Sep_Toil_4')

In [ ]:
answer = pd.concat([sep_1, sep_2, sep_3, sep_4, toil_df.loc[:, 'Совмещенный (1)':]], axis = 1)
answer.rename(columns = ({'Совмещенный (1)' : 'Comb_Toil_1', 'Совмещенный (2)' : 'Comb_Toil_2',
                          'Совмещенный (3)' : 'Comb_Toil_3', 'Совмещенный (4)' : 'Comb_Toil_4',}), inplace = True)

In [ ]:
df_egor = pd.concat([df_metro, price_df, dop_df, answer], axis = 1)
df = pd.concat([df, df_egor], axis =1)
df

,index,ID объявления,Количество комнат,Тип,Метро,Адрес,"Площадь, м2",Дом,parking_lot,Цена,...,Telephone,Fridge,Sep_Toil_1,Sep_Toil_2,Sep_Toil_3,Sep_Toil_4,Comb_Toil_1,Comb_Toil_2,Comb_Toil_3,Comb_Toil_4
0,0,271271157,4,Квартира,"[м. Смоленская , 9 мин пешком]","Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",1,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",...,1,1,0,0,0,0,0,0,0,0
1,1,271634126,4,Квартира,"[м. Смоленская , 8 мин пешком]","Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",1,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",...,0,1,1,0,0,0,0,1,0,0
2,2,271173086,"4, Оба варианта",Квартира,"[м. Смоленская , 7 мин пешком]","Москва, улица Новый Арбат, 27",200.0/116.0/4.0,5/16,1,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",...,1,1,0,0,0,0,0,0,1,0
3,3,272197456,"4, Оба варианта",Квартира,"[м. Смоленская , 3 мин пешком]","Москва, переулок Плотников, 21С1",170.0/95.0/17.0,5/6,1,"400000.0 руб./ За месяц, Залог - 400000 руб., ...",...,1,1,0,0,0,0,0,0,1,0
4,4,273614615,2,Квартира,"[м. Арбатская , 7 мин пешком]","Москва, улица Новый Арбат, 15",58.0/38.0/5.0,"12/26, Панельный",0,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19732,23363,215565511,NaN,Квартира,"[м. Говорово , 8 мин пешком]","Москва, Боровское шоссе, 2к7, ш. Боровское (3 ...",35.0/16.4/8.0,10/14,0,"42000.0 руб./ За месяц, Залог - 42000 руб., Ко...",...,0,1,0,0,0,0,1,0,0,0
19733,23364,274654844,1,Квартира,"[м. Солнцево , 7 мин пешком]","Москва, Производственная улица, 8к1, ш. Боровс...",38.7/16.5/11.0,"5/18, Монолитный",0,"45000.0 руб./ За месяц, Залог - 45000 руб., Ко...",...,0,1,0,0,0,0,1,0,0,0
19734,23365,268679909,"2, Оба варианта",Квартира,"[м. Солнцево , 6 мин пешком]","Москва, Боровский проезд, 11",43.1,"5/5, Кирпичный",0,"50000.0 руб./ За месяц, Залог - 50000 руб., Ко...",...,0,1,0,0,0,0,1,0,0,0
19735,23366,274807525,2,Квартира,"[м. Солнцево , 11 мин пешком]","Москва, улица Богданова, 6к1, ш. Боровское (3 ...",52.5/10.0,"8/23, Монолитный",1,"55000.0 руб./ За месяц, Залог - 50000 руб., Ко...",...,0,1,1,0,0,0,1,0,0,0


In [ ]:
df.drop(columns=['index', 'Количество комнат', 'Type', 'Метро', 'Адрес',
       'Площадь, м2', 'Дом', 'Цена', 'Описание',
       'Площадь комнат, м2', 'Санузел', 'Можно с детьми/животными',
       'Дополнительно', 'Название ЖК', 'Серия дома', 'Ссылка на объявление',
       'house_type', 'phone_1', 'phone_2'], inplace = True)

In [41]:
df.columns

Index(['index', 'ID  объявления', 'Количество комнат', 'Тип', 'Метро', 'Адрес',
       'Площадь, м2', 'Дом', 'Парковка', 'Цена', 'Телефоны', 'Описание',
       'Ремонт', 'Площадь комнат, м2', 'Балкон', 'Окна', 'Санузел',
       'Можно с детьми/животными', 'Дополнительно', 'Название ЖК',
       'Серия дома', 'Высота потолков, м', 'Лифт', 'Мусоропровод',
       'Ссылка на объявление'],
      dtype='object')

In [42]:
df.to_csv('data_finished.csv')